# What are Recurrent Neural Networks


The idea behind RNNs is to make use of sequential information. In a traditional neural network we assume that all inputs (and outputs) are independent of each other. But for many tasks that’s a very bad idea. If you want to predict the next word in a sentence you better know which words came before it. RNNs are called recurrent because they perform the same task for every element of a sequence, with the output being depended on the previous computations. Another way to think about RNNs is that they have a “memory” which captures information about what has been calculated so far. In theory RNNs can make use of information in arbitrarily long sequences, but in practice they are limited to looking back only a few steps (more on this later). Here is what a typical RNN looks like:

![](http://d3kbpzbmcynnmx.cloudfront.net/wp-content/uploads/2015/09/rnn.jpg)

The above diagram shows a RNN being unrolled (or unfolded) into a full network. By unrolling we simply mean that we write out the network for the complete sequence. For example, if the sequence we care about is a sentence of 5 words, the network would be unrolled into a 5-layer neural network, one layer for each word. The formulas that govern the computation happening in a RNN are as follows:

   * x_t is the input at time step t. For example, x_1 could be a one-hot vector corresponding to the second word of a sentence.
   * s_t is the hidden state at time step t. It’s the “memory” of the network. s_t is calculated based on the previous hidden state and the input at the current step: s_t=f(Ux_t + Ws_{t-1}). The function f usually is a nonlinearity such as tanh or ReLU.  s_{-1}, which is required to calculate the first hidden state, is typically initialized to all zeroes.
   * o_t is the output at step t. For example, if we wanted to predict the next word in a sentence it would be a vector of probabilities across our vocabulary. o_t = \mathrm{softmax}(Vs_t).

There are a few things to note here:

   * You can think of the hidden state s_t as the memory of the network. s_t captures information about what happened in all the previous time steps. The output at step o_t is calculated solely based on the memory at time t. As briefly mentioned above, it’s a bit more complicated  in practice because s_t typically can’t capture information from too many time steps ago.
   * Unlike a traditional deep neural network, which uses different parameters at each layer, a RNN shares the same parameters (U, V, W above) across all steps. This reflects the fact that we are performing the same task at each step, just with different inputs. This greatly reduces the total number of parameters we need to learn.

   * The above diagram has outputs at each time step, but depending on the task this may not be necessary. For example, when predicting the sentiment of a sentence we may only care about the final output, not the sentiment after each word. Similarly, we may not need inputs at each time step. The main feature of an RNN is its hidden state, which captures some information about a sequence.


# Anna KaRNNa

We will build a character-wise RNN trained on Anna Karenina. It'll be able to generate new text based on the text from the book.

<img src="http://karpathy.github.io/assets/rnn/charseq.jpeg" width="500">

In [15]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [24]:
vocab_to_int

{'3': 0,
 '*': 1,
 't': 2,
 'w': 3,
 'V': 4,
 'f': 5,
 ',': 6,
 '4': 7,
 'K': 8,
 '_': 9,
 '8': 10,
 'T': 11,
 'q': 12,
 'A': 13,
 'Q': 14,
 '`': 15,
 'I': 16,
 'J': 17,
 'z': 18,
 's': 19,
 'W': 20,
 ' ': 21,
 'e': 22,
 'M': 23,
 'F': 24,
 'i': 25,
 'h': 26,
 'o': 27,
 'P': 28,
 'U': 29,
 '-': 30,
 '"': 31,
 'B': 32,
 '6': 33,
 'G': 34,
 'O': 35,
 '%': 36,
 "'": 37,
 '7': 38,
 'v': 39,
 '0': 40,
 'N': 41,
 ')': 42,
 'E': 43,
 'L': 44,
 'j': 45,
 'C': 46,
 'S': 47,
 '\n': 48,
 '!': 49,
 'd': 50,
 'a': 51,
 'D': 52,
 'g': 53,
 'x': 54,
 ';': 55,
 '2': 56,
 'H': 57,
 ':': 58,
 'y': 59,
 '.': 60,
 'k': 61,
 'l': 62,
 'Z': 63,
 '9': 64,
 '5': 65,
 'c': 66,
 'Y': 67,
 '$': 68,
 'r': 69,
 '1': 70,
 'R': 71,
 'n': 72,
 'X': 73,
 '@': 74,
 'b': 75,
 'm': 76,
 'p': 77,
 'u': 78,
 '/': 79,
 '(': 80,
 '&': 81,
 '?': 82}

In [16]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [25]:
chars

array([46, 26, 51, ..., 19, 60, 48])

In [21]:
chars

array([46, 26, 51, ..., 19, 60, 48])

In [18]:
vocab_to_int

{'3': 0,
 '*': 1,
 't': 2,
 'w': 3,
 'V': 4,
 'f': 5,
 ',': 6,
 '4': 7,
 'K': 8,
 '_': 9,
 '8': 10,
 'T': 11,
 'q': 12,
 'A': 13,
 'Q': 14,
 '`': 15,
 'I': 16,
 'J': 17,
 'z': 18,
 's': 19,
 'W': 20,
 ' ': 21,
 'e': 22,
 'M': 23,
 'F': 24,
 'i': 25,
 'h': 26,
 'o': 27,
 'P': 28,
 'U': 29,
 '-': 30,
 '"': 31,
 'B': 32,
 '6': 33,
 'G': 34,
 'O': 35,
 '%': 36,
 "'": 37,
 '7': 38,
 'v': 39,
 '0': 40,
 'N': 41,
 ')': 42,
 'E': 43,
 'L': 44,
 'j': 45,
 'C': 46,
 'S': 47,
 '\n': 48,
 '!': 49,
 'd': 50,
 'a': 51,
 'D': 52,
 'g': 53,
 'x': 54,
 ';': 55,
 '2': 56,
 'H': 57,
 ':': 58,
 'y': 59,
 '.': 60,
 'k': 61,
 'l': 62,
 'Z': 63,
 '9': 64,
 '5': 65,
 'c': 66,
 'Y': 67,
 '$': 68,
 'r': 69,
 '1': 70,
 'R': 71,
 'n': 72,
 'X': 73,
 '@': 74,
 'b': 75,
 'm': 76,
 'p': 77,
 'u': 78,
 '/': 79,
 '(': 80,
 '&': 81,
 '?': 82}

In [26]:
vocab_to_int

{'3': 0,
 '*': 1,
 't': 2,
 'w': 3,
 'V': 4,
 'f': 5,
 ',': 6,
 '4': 7,
 'K': 8,
 '_': 9,
 '8': 10,
 'T': 11,
 'q': 12,
 'A': 13,
 'Q': 14,
 '`': 15,
 'I': 16,
 'J': 17,
 'z': 18,
 's': 19,
 'W': 20,
 ' ': 21,
 'e': 22,
 'M': 23,
 'F': 24,
 'i': 25,
 'h': 26,
 'o': 27,
 'P': 28,
 'U': 29,
 '-': 30,
 '"': 31,
 'B': 32,
 '6': 33,
 'G': 34,
 'O': 35,
 '%': 36,
 "'": 37,
 '7': 38,
 'v': 39,
 '0': 40,
 'N': 41,
 ')': 42,
 'E': 43,
 'L': 44,
 'j': 45,
 'C': 46,
 'S': 47,
 '\n': 48,
 '!': 49,
 'd': 50,
 'a': 51,
 'D': 52,
 'g': 53,
 'x': 54,
 ';': 55,
 '2': 56,
 'H': 57,
 ':': 58,
 'y': 59,
 '.': 60,
 'k': 61,
 'l': 62,
 'Z': 63,
 '9': 64,
 '5': 65,
 'c': 66,
 'Y': 67,
 '$': 68,
 'r': 69,
 '1': 70,
 'R': 71,
 'n': 72,
 'X': 73,
 '@': 74,
 'b': 75,
 'm': 76,
 'p': 77,
 'u': 78,
 '/': 79,
 '(': 80,
 '&': 81,
 '?': 82}

Let's check out the first 100 characters, make sure everything is peachy.

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

And we can see the characters encoded as integers.

In [4]:
chars[:100]

array([46, 26, 51, 77,  2, 22, 69, 21, 70, 48, 48, 48, 57, 51, 77, 77, 59,
       21,  5, 51, 76, 25, 62, 25, 22, 19, 21, 51, 69, 22, 21, 51, 62, 62,
       21, 51, 62, 25, 61, 22, 55, 21, 22, 39, 22, 69, 59, 21, 78, 72, 26,
       51, 77, 77, 59, 21,  5, 51, 76, 25, 62, 59, 21, 25, 19, 21, 78, 72,
       26, 51, 77, 77, 59, 21, 25, 72, 21, 25,  2, 19, 21, 27,  3, 72, 48,
        3, 51, 59, 60, 48, 48, 43, 39, 22, 69, 59,  2, 26, 25, 72])

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
np.max(chars)+1

83

## Making training and validation batches

Now I need to split up the data into batches, and into training and validation sets. I should be making a test set here, but I'm not going to worry about that. My test will be if the network can generate new text.

Here I'll make both input and target arrays. The targets are the same as the inputs, except shifted one character over. I'll also drop the last bit of data so that I'll only have completely full batches.

The idea here is to make a 2D matrix where the number of rows is equal to the batch size. Each row will be one long concatenated string from the character data. We'll split this data into a training set and validation set using the `split_frac` keyword. This will keep 90% of the batches in the training set, the other 10% in the validation set.

In [6]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps: Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    
    Returns train_x, train_y, val_x, val_y
    """
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches*slice_size]
    y = chars[1: n_batches*slice_size + 1]
    
    # Split the data into batch_size slices, then stack them into a 2D matrix 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch_size x n_batches*num_steps
    
    # Split into training and validation sets, keep the first split_frac batches for training
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [27]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 50)

In [29]:
train_x

array([[46, 26, 51, ..., 21, 76, 22],
       [21, 51, 76, ..., 19,  2, 51],
       [39, 25, 72, ..., 51, 59, 60],
       ...,
       [55, 21, 75, ..., 27, 39, 25],
       [ 2, 21, 25, ..., 19,  2, 51],
       [21, 19, 51, ..., 77, 78,  2]])

In [28]:
train_x.shape

(10, 178650)

Looking at the size of this array, we see that we have rows equal to the batch size. When we want to get a batch out of here, we can grab a subset of this array that contains all the rows but has a width equal to the number of steps in the sequence. The first batch looks like this:

In [9]:
train_x[:,:50]

array([[46, 26, 51, 77,  2, 22, 69, 21, 70, 48, 48, 48, 57, 51, 77, 77,
        59, 21,  5, 51, 76, 25, 62, 25, 22, 19, 21, 51, 69, 22, 21, 51,
        62, 62, 21, 51, 62, 25, 61, 22, 55, 21, 22, 39, 22, 69, 59, 21,
        78, 72],
       [21, 51, 76, 21, 72, 27,  2, 21, 53, 27, 25, 72, 53, 21,  2, 27,
        21, 19,  2, 51, 59,  6, 31, 21, 51, 72, 19,  3, 22, 69, 22, 50,
        21, 13, 72, 72, 51,  6, 21, 19, 76, 25, 62, 25, 72, 53,  6, 21,
        75, 78],
       [39, 25, 72, 60, 48, 48, 31, 67, 22, 19,  6, 21, 25,  2, 37, 19,
        21, 19, 22,  2,  2, 62, 22, 50, 60, 21, 11, 26, 22, 21, 77, 69,
        25, 66, 22, 21, 25, 19, 21, 76, 51, 53, 72, 25,  5, 25, 66, 22,
        72,  2],
       [72, 21, 50, 78, 69, 25, 72, 53, 21, 26, 25, 19, 21, 66, 27, 72,
        39, 22, 69, 19, 51,  2, 25, 27, 72, 21,  3, 25,  2, 26, 21, 26,
        25, 19, 48, 75, 69, 27,  2, 26, 22, 69, 21,  3, 51, 19, 21,  2,
        26, 25],
       [21, 25,  2, 21, 25, 19,  6, 21, 19, 25, 69, 49, 31, 21, 19, 

I'll write another function to grab batches out of the arrays made by `split_data`. Here each batch will be a sliding window on these arrays with size `batch_size X num_steps`. For example, if we want our network to train on a sequence of 100 characters, `num_steps = 100`. For the next batch, we'll shift this window the next sequence of `num_steps` characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [10]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size/num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

## Building the model

Below is a function where I build the graph for the network.

In [35]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
    
    # When we're using this network for sampling later, we'll be passing in
    # one character at a time, so providing an option for that
    if sampling == True:
        batch_size, num_steps = 1, 1

    tf.reset_default_graph()
    
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # One-hot encoding the input and target characters
    x_one_hot = tf.one_hot(inputs, num_classes)
    y_one_hot = tf.one_hot(targets, num_classes)

    ### Build the RNN layers
    # Use a basic LSTM cell
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    # Add dropout to the cell
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    # Stack up multiple LSTM layers, for deep learning
#     stacked_rnn = []
#         for iiLyr in range():
#             stacked_rnn.append(tf.nn.rnn_cell.LSTMCell(num_units=lstm_size, state_is_tuple=True))
    
    #cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    initial_state = lstm.zero_state(batch_size, tf.float32)

    ### Run the data through the RNN layers
    # This makes a list where each element is on step in the sequence
    rnn_inputs = [tf.squeeze(i, squeeze_dims=[1]) for i in tf.split(x_one_hot, num_steps, 1)]
    
    # Run each sequence step through the RNN and collect the outputs
    outputs, state = tf.contrib.rnn.static_rnn(lstm, rnn_inputs, initial_state=initial_state)
    final_state = state
    
    # Reshape output so it's a bunch of rows, one output row for each step for each batch
    seq_output = tf.concat(outputs, axis=1)
    output = tf.reshape(seq_output, [-1, lstm_size])
    
    # Now connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(num_classes))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and batch
    logits = tf.matmul(output, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    preds = tf.nn.softmax(logits, name='predictions')
    
    # Reshape the targets to match the logits
    y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    cost = tf.reduce_mean(loss)

    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    # Export the nodes
    # NOTE: I'm using a namedtuple here because I think they are cool
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## Hyperparameters

Here I'm defining the hyperparameters for the network. 

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to write it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.


In [36]:
batch_size = 100
num_steps = 100 
lstm_size = 512
num_layers = 2
learning_rate = 0.001
keep_prob = 10.5

In [37]:
!mkdir checkpoints

A subdirectory or file checkpoints already exists.


## Training

Time for training which is pretty straightforward. Here I pass in some data, and get an LSTM state back. Then I pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I calculate the validation loss and save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}_v{validation loss}.ckpt`

In [ ]:
epochs = 1
# Save every N iterations
save_every_n = 200
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)

In [40]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.cost, model.final_state, model.optimizer], 
                                                 feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/b),
                  '{:.4f} sec/batch'.format((end-start)))
        
            
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    batch_loss, new_state = sess.run([model.cost, model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)

                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints/i{}_l{}_v{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))

Epoch 1/20  Iteration 1/3560 Training loss: 4.4241 0.4827 sec/batch
Epoch 1/20  Iteration 2/3560 Training loss: 4.3984 0.1415 sec/batch
Epoch 1/20  Iteration 3/3560 Training loss: 4.3531 0.1861 sec/batch
Epoch 1/20  Iteration 4/3560 Training loss: 4.1896 0.0971 sec/batch
Epoch 1/20  Iteration 5/3560 Training loss: 4.0333 0.0980 sec/batch
Epoch 1/20  Iteration 6/3560 Training loss: 3.9268 0.2053 sec/batch
Epoch 1/20  Iteration 7/3560 Training loss: 3.8355 0.1948 sec/batch
Epoch 1/20  Iteration 8/3560 Training loss: 3.7560 0.1085 sec/batch
Epoch 1/20  Iteration 9/3560 Training loss: 3.6924 0.1978 sec/batch
Epoch 1/20  Iteration 10/3560 Training loss: 3.6394 0.1140 sec/batch
Epoch 1/20  Iteration 11/3560 Training loss: 3.5931 0.1870 sec/batch
Epoch 1/20  Iteration 12/3560 Training loss: 3.5547 0.0970 sec/batch
Epoch 1/20  Iteration 13/3560 Training loss: 3.5219 0.0990 sec/batch
Epoch 1/20  Iteration 14/3560 Training loss: 3.4946 0.1030 sec/batch
Epoch 1/20  Iteration 15/3560 Training loss

Epoch 1/20  Iteration 120/3560 Training loss: 2.9521 0.1920 sec/batch
Epoch 1/20  Iteration 121/3560 Training loss: 2.9489 0.0980 sec/batch
Epoch 1/20  Iteration 122/3560 Training loss: 2.9454 0.1000 sec/batch
Epoch 1/20  Iteration 123/3560 Training loss: 2.9420 0.1077 sec/batch
Epoch 1/20  Iteration 124/3560 Training loss: 2.9386 0.1953 sec/batch
Epoch 1/20  Iteration 125/3560 Training loss: 2.9352 0.1896 sec/batch
Epoch 1/20  Iteration 126/3560 Training loss: 2.9316 0.0880 sec/batch
Epoch 1/20  Iteration 127/3560 Training loss: 2.9283 0.1100 sec/batch
Epoch 1/20  Iteration 128/3560 Training loss: 2.9251 0.0990 sec/batch
Epoch 1/20  Iteration 129/3560 Training loss: 2.9219 0.0880 sec/batch
Epoch 1/20  Iteration 130/3560 Training loss: 2.9193 0.0890 sec/batch
Epoch 1/20  Iteration 131/3560 Training loss: 2.9160 0.1240 sec/batch
Epoch 1/20  Iteration 132/3560 Training loss: 2.9129 0.0980 sec/batch
Epoch 1/20  Iteration 133/3560 Training loss: 2.9098 0.0910 sec/batch
Epoch 1/20  Iteratio

Epoch 2/20  Iteration 237/3560 Training loss: 2.2763 0.1010 sec/batch
Epoch 2/20  Iteration 238/3560 Training loss: 2.2758 0.0870 sec/batch
Epoch 2/20  Iteration 239/3560 Training loss: 2.2746 0.1100 sec/batch
Epoch 2/20  Iteration 240/3560 Training loss: 2.2739 0.0890 sec/batch
Epoch 2/20  Iteration 241/3560 Training loss: 2.2732 0.0870 sec/batch
Epoch 2/20  Iteration 242/3560 Training loss: 2.2722 0.1260 sec/batch
Epoch 2/20  Iteration 243/3560 Training loss: 2.2712 0.0890 sec/batch
Epoch 2/20  Iteration 244/3560 Training loss: 2.2705 0.1110 sec/batch
Epoch 2/20  Iteration 245/3560 Training loss: 2.2694 0.1000 sec/batch
Epoch 2/20  Iteration 246/3560 Training loss: 2.2681 0.1173 sec/batch
Epoch 2/20  Iteration 247/3560 Training loss: 2.2669 0.1101 sec/batch
Epoch 2/20  Iteration 248/3560 Training loss: 2.2662 0.1771 sec/batch
Epoch 2/20  Iteration 249/3560 Training loss: 2.2655 0.2032 sec/batch
Epoch 2/20  Iteration 250/3560 Training loss: 2.2647 0.1920 sec/batch
Epoch 2/20  Iteratio

Epoch 2/20  Iteration 356/3560 Training loss: 2.1849 0.0980 sec/batch
Epoch 3/20  Iteration 357/3560 Training loss: 2.1833 0.0990 sec/batch
Epoch 3/20  Iteration 358/3560 Training loss: 2.1190 0.1000 sec/batch
Epoch 3/20  Iteration 359/3560 Training loss: 2.1067 0.1020 sec/batch
Epoch 3/20  Iteration 360/3560 Training loss: 2.0973 0.0990 sec/batch
Epoch 3/20  Iteration 361/3560 Training loss: 2.0907 0.1010 sec/batch
Epoch 3/20  Iteration 362/3560 Training loss: 2.0824 0.1010 sec/batch
Epoch 3/20  Iteration 363/3560 Training loss: 2.0809 0.0890 sec/batch


KeyboardInterrupt: 

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [41]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints\\i200_l512_v2.294.ckpt"
all_model_checkpoint_paths: "checkpoints\\i200_l512_v2.342.ckpt"
all_model_checkpoint_paths: "checkpoints\\i400_l512_v2.084.ckpt"
all_model_checkpoint_paths: "checkpoints\\i600_l512_v1.915.ckpt"
all_model_checkpoint_paths: "checkpoints\\i800_l512_v1.800.ckpt"
all_model_checkpoint_paths: "checkpoints\\i1000_l512_v1.697.ckpt"
all_model_checkpoint_paths: "checkpoints\\i200_l512_v2.294.ckpt"

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [43]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [44]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [51]:
# Change the name of latest checkpoint accordingly
checkpoint = "checkpoints\\i1000_l512_v1.697.ckpt"
samp = sample(checkpoint, 100, lstm_size, len(vocab), prime="sudh")
print(samp)

INFO:tensorflow:Restoring parameters from checkpoints\i1000_l512_v1.697.ckpt
sudher him an the paider and
then the she her hem are had been.

She wanksed hin ellass and as and house.
